In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import scale

## Load Data

In [2]:
df_movies = pd.read_csv('./movies_metadata_clean.csv', encoding='ISO-8859-1')
df_users = pd.read_csv('./movies_ratings_small.csv', encoding='ISO-8859-1')

## Build recommendations

1. content based filtering

In [3]:
df_movies.shape

# Remove duplicates
df_movies.drop_duplicates(subset='title',keep='first',inplace=True)
df_movies.shape

(45463, 24)

(42277, 24)

In [4]:
# Prepare description column

df_movies['tagline'] = df_movies['tagline'].fillna('')
df_movies['description'] = df_movies['overview'] + df_movies['tagline']
df_movies['description'] = df_movies['description'].fillna('')
df_movies['description']

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
45456    It's the year 3000 AD. The world's most danger...
45458    Rising and falling between a man and woman.Ris...
45459    An artist struggles to finish his work while a...
45461    In a small town live two brothers, one a minis...
45462    50 years after decriminalisation of homosexual...
Name: description, Length: 42277, dtype: object

## Build model

In [5]:
## Generate a matrix of common terms that show up in each movie

from sklearn.feature_extraction.text import TfidfVectorizer

model = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')

tfidf_matrix = model.fit_transform(df_movies['description'])
tfidf_matrix.shape

(42277, 1047434)

In [6]:
# Calculate the cosine similarity between each pair of movies

from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix,tfidf_matrix)
cosine_sim.shape

(42277, 42277)

## Generate Recommendation

In [17]:
titles = df_movies['title']
indicies = pd.Series(df_movies.index, index=df_movies['title'])

def get_recommendations(title):
    idx = indicies[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indicies = [i[0] for i in sim_scores]
    return titles.iloc[movie_indicies]

In [18]:
get_recommendations('The Godfather')

44027    The Godfather Trilogy: 1972-1990
1178               The Godfather: Part II
31971                    Honor Thy Father
23125                          Blood Ties
38027            A Mother Should Be Loved
18322                     The Outside Man
11297                    Household Saints
4324                                 Made
5433                   Johnny Dangerously
18224                           Miss Bala
Name: title, dtype: object

In [19]:
get_recommendations('Jumanji')

21632                  Table No. 21
9503                      Word Wars
43124                       The Bar
8801                        Quintet
17223                The Dark Angel
37444            The Ouija Exorcism
15512               Le Pont du Nord
34771    Doctor Who: Last Christmas
44373             Liar Game: Reborn
35507                      The Mend
Name: title, dtype: object

2. Collaborative Filtering

In [21]:
df_users.tail(10)

,userId,movieId,rating,timestamp
99994,671,5952,5.0,1063502716
99995,671,5989,4.0,1064890625
99996,671,5991,4.5,1064245387
99997,671,5995,4.0,1066793014
99998,671,6212,2.5,1065149436
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663
100003,671,6565,3.5,1074784724


In [1]:
# Prepare data into Surprise Library format

from surprise import Dataset
from surprise import Reader

# reader = Reader(rating_scale=(1,5))
reader = Reader()
data = Dataset.load_from_df(df_users[['userId','movieId', 'rating']], reader)


ModuleNotFoundError: No module named 'surprise'

In [ ]:
from surprise.model_selection import train_test_split

x_train, x_test = train_test_split(x, test_size=.25)

In [ ]:
# Define SVD model

from surprise import SVD
model_svd_rating = SVD()

In [ ]:
# Fit SVD model

model_svd_rating.fit(x_train)
tests_pred = model_svd_rating.test(x_test)

In [ ]:
# Evaluate SVD accuracy

from surprise import accuracy

accuracy.rmse(tests_pred)

In [ ]:
## Cross-validate

from surprise.model_selection import cross_validate

cross_validate(model_svd_rating, x, measures=['RMSE','MAE'], cv=5, verbose=True)

In [ ]:
# Predict rating for new movie

df_users[df_users['userId']] = 

In [ ]:
# Predict what user 1 would have rated movie 302

model_svd_rating.predict(1, 302)